In [ ]:
using Plots # mega długo się ładuje
using LaTeXStrings
using Distributions

# Task 1

In [ ]:
include("alphastable.jl")

In [ ]:
S0 = alphastable(10 ^ 5, 1, 1.8, 0.5, 1, 0, 0);
S1 = alphastable(10 ^ 5, 1, 1.8, 0.5, 1, 0, 1);
histogram(S0, xlim = [-5,5], ylim = [0,0.35], legend = false, normalize = true, reuse = false)
histogram!(S1, xlim = [-5,5], ylim = [0,0.35], legend = false, normalize = true, reuse = false, alpha = 0.5)

# Task 2


## Writing the empirical cdf function




In [ ]:
function ECDF(X)
    n = length(X);
    x_values = sort(vec(X));
    y_values = (1:n) ./ n;
    return [x_values, y_values];
end

## Testing the function on the exponential and normal distributions

In [ ]:
M = 10 ^ 6;
lambda = 1;

In [ ]:
X = vec(d_exp(1, M, lambda));
x_values, y_values = ECDF(X);

In [ ]:
plot(x_values, y_values, reuse = false, labels = L"empirical")
plot!(x_values, 1 .- exp.(-lambda * x_values), labels = L"theoretical", title = L"CDF", xlabel = L"x", ylabel = L"\Phi(x)")

In [ ]:
my_dist = Normal(0,1);
t = range(-3.5, 3.5, length = M);
X = randn(M);
x_values, y_values = ECDF(X);
plot(x_values, y_values, labels = L"empirical")
plot!(xlabel = L"x",
    ylabel = L"\Phi(x)", title  = L"CDF",
    labels = L"theoretical",
    t -> cdf(my_dist, t), t[1], t[end], linecolor = colorant"magenta")

## Tails functionality

In [ ]:
tail_X = 1 .- y_values;
threshold = 3;
t = threshold:0.01:maximum(x_values[x_values .> threshold]);
plot(x_values[x_values .> threshold], log.(tail_X[x_values .> threshold]), legend = false, reuse = false)
plot!(plot!(xlabel = L"x",
    ylabel = L"\Phi(x)", title  = L"CDF",
    labels = L"theoretical",
    t -> log.(1 - cdf(my_dist, t)), threshold, maximum(x_values[x_values .> threshold]),
    linecolor = colorant"magenta"))

## CDF comparison for $\alpha$-stable

In [ ]:
alp = 1;
beta = 0;
gam = 1;
delta = 1;

In [ ]:
MC = 10 ^ 7;
X = alphastable(MC, 1, alp, beta, gam, delta, 1);
t = float.(0:1:100);
lambda = 1/2;
c = 1 / 2;

In [ ]:
tali_X = CDF(-t, vec(X), MC);

plot(-t, tali_X, reuse = false, labels = "empirical tail", yaxis=:log)
plot!(-t, exp.(- lambda .* t), labels = L"\exp(-\lambda t)", yaxis=:log)
plot!(-t, c .* t .^ (-alp), labels = L"c t^{-\alpha}", yaxis=:log, legend=:bottomright)

## Tails comparison for $\alpha$-stable

In [ ]:
alp = 1;
beta = 0;
gam = 1;
delta = 1;

In [ ]:
MC = 10 ^ 7;
X = alphastable(MC, 1, alp, beta, gam, delta, 1);
t = float.(0:1:100);
lambda = 1/2;
c = 1 / 2;

In [ ]:
tali_X = 1 .- CDF(t, vec(X), MC);

plot(t, tali_X, reuse = false, labels = "empirical tail", yaxis=:log)
plot!(t, exp.(- lambda .* t), labels = L"\exp(-\lambda t)", yaxis=:log)
plot!(t, c .* t .^ (-alp), labels = L"c t^{-\alpha}", yaxis=:log, legend=:bottomleft)

# Task 3

In [ ]:
function characterist_r_i(t, X, MC)
    len = length(t);
    X = X * ones(1,length(t));
    t = t' .* ones(MC, 1);
    Re = (sum(cos.(X .* t), dims = 1) ./ (MC * ones(1, len)))';
    Im = (sum(sin.(X .* t), dims = 1) ./ (MC * ones(1, len)))';
    return [Re Im];
end

function d_exp(N, M, lambda)
    return -1 / lambda * log.(rand(N, M));
end

In [ ]:
N = 10 ^ 5;
lambda3 = 2;
X = d_exp(N, 1, lambda3);
t = 0:0.05:10;


f_re3(t) = lambda3 ^ 2 ./ (lambda3 ^ 2 .+ t .^ 2);
f_im3(t) = lambda3 * t ./ (lambda3 ^ 2 .+ t .^ 2);
FunChar = characterist_r_i(t, X, N);

In [ ]:

plot(t, FunChar[:,1], linewidth = 5, labels = L"\textrm{Numerical }\Re(\psi_X(t))", reuse = false)
plot!(t, f_re3(t), linewidth = 3, labels = L"\textrm{Teoretical }\Re(\psi_X(t))")


In [ ]:

plot(t, FunChar[:,2], linewidth = 5, labels = L"\textrm{Numerical }\Im(\psi_X(t))", reuse = false)
plot!(t, f_im3(t), linewidth = 3, labels = L"\textrm{Teoretical }\Im(\psi_X(t))")

# Task 4

In [ ]:
function running_moment(sample, tau)
    repeat(sample, outer = [1, length(X)])
    M = [ x<=y ? 1 : (x == y ? 0 : 0) for x in 1:length(sample), y in 1:length(sample)]
    Y = (M .* sample) .^ tau
    return sum(Y, dims = 1) ./ (1:length(sample))'
end

In [ ]:
alp = 1.8;
beta = 0;
gam = 1;
delta = 1;
MC = 10 ^ 4;
X = alphastable(MC, 1, alp, beta, gam, delta, 1);

In [ ]:
ps = [];
for tau in range(1, 4)
    push!(ps, plot(vec(running_moment(X, tau)), reuse = false, title = string(L"\tau", " = ", "$tau")));
end

In [ ]:
plot(ps...) # splatting

# Task 5

In [ ]:
N = 10 ^ 5;
lambda5 = 2;
X = d_exp(N, 1, lambda5);
t = 0:0.01:2.5;


f_pdf6(t) = lambda5 .* exp.(- lambda5 .* t);

In [ ]:
function CDF(t, X, MC)
    len = length(t);
    X = X * ones(1, length(t));
    return (sum(X .< t' .* ones(MC,1), dims = 1) ./ (MC * ones(1,len)))';
end

function PDF(t, X, MC)
    dt = (t[2] - t[1])
    t = t .- dt / 2;
    p = CDF(t[1:end-1], X, MC);
    q = CDF(t[2:end], X, MC);
    return (.- p .+ q) ./ dt;
end

In [ ]:

plot(t, f_pdf6(t), labels = "teoretical pdf Exp(2)", linewidth = 5, reuse = false)
plot!(t[1:end-1], PDF(t, X, N), labels = "numerical pdf Exp(2)", linewidth = 2)
